In [ ]:
import logging

logger = logging.getLogger(__name__)

import zipfile
from pathlib import Path

import geopandas as gpd
import pandas as pd
from _helpers import configure_logging

if __name__ == "__main__":

    configure_logging(snakemake)

    # Move downloaded zip file to the right output folder
    # Snakemake.input
    Path(snakemake.input[0]).rename(snakemake.output["zip"])

    logging.info(f"Extracting archive: {snakemake.output['zip']}")
    # WDPA files are twofold zipped: the parent folder (unzipping here)...
    with zipfile.ZipFile(snakemake.output["zip"], "r") as zf:
        zf.extractall(snakemake.output["folder"])

    # ... and then combine the different zipped layers
    layers = []
    layer_files = list(Path(snakemake.output["folder"]).glob("WDPA*.zip"))

    logging.info(
        f"Found {len(layer_files)} WDPA layers in {snakemake.output['folder']}"
    )
    for layer_file in layer_files:
        logging.info(f"Opening file {str(layer_file)}...")
        layers.append(gpd.read_file("zip://" + str(layer_file)))

    logging.info("Saving and combining WDPA layers.")
    gdf = gpd.GeoDataFrame(pd.concat(layers), crs=layers[0].crs)
    gdf.to_file(snakemake.output["gpkg"], driver="GPKG")